This Python script reads in a tab-delimited *.txt of the top 4600 Spotify songs of 2024 and adds title, album, artist, release date, ranking, and stream count to a sqlite DB as three tables. Note that splitting on '\t' results in a cleaner DB output compared to splitting on ',' from a *.csv. [Source data here](https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024?select=Most+Streamed+Spotify+Songs+2024.csv)

In [1]:
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    release_date TEXT, rank_2024 INTEGER, spotify_streams INTEGER
);
''')

handle = open('spotify2024.txt')


for line in handle:
    line = line.strip();
    pieces = line.split('\t')
    if len(pieces) < 6 : continue

    track = pieces[0]
    album = pieces[1]
    artist = pieces[2]
    release_date = pieces[3]
    rank_2024 = pieces[5]
    spotify_streams = pieces[7]
    
    print(track, album, artist, release_date, rank_2024, spotify_streams)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, release_date, rank_2024, spotify_streams) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        ( track, album_id, release_date, rank_2024, spotify_streams ) )

    conn.commit()
   

MILLION DOLLAR BABY Million Dollar Baby - Single Tommy Richman 4/26/2024 1 390470936
Not Like Us Not Like Us Kendrick Lamar 5/4/2024 2 323703884
i like the way you kiss me I like the way you kiss me Artemas 3/19/2024 3 601309283
Flowers Flowers - Single Miley Cyrus 1/12/2023 4 2031280633
Houdini Houdini Eminem 5/31/2024 5 107034922
Lovin On Me Lovin On Me Jack Harlow 11/10/2023 6 670665438
Beautiful Things Beautiful Things Benson Boone 1/18/2024 7 900158751
Gata Only Gata Only FloyyMenor 2/2/2024 8 675079153
Danza Kuduro - Cover ýýýýýýýýýýýýýýýýýýýýý - ýýýýýýýýýýýýýýýýýý - MUSIC LAB JPN 6/9/2024 9 1653018119
BAND4BAND (feat. Lil Baby) BAND4BAND (feat. Lil Baby) Central Cee 5/23/2024 10 90676573
I Had Some Help (feat. Morgan Wallen) I Had Some Help Post Malone 5/10/2024 11 250080413
The Door The Door Teddy Swims 6/14/2024 12 111155129
LUNCH HIT ME HARD AND SOFT Billie Eilish 5/17/2024 13 221636195
Like That WE DON'T TRUST YOU Future 3/22/2024 14 355186436
bathroom floor bathroom floor K